In [3]:
from dotenv import load_dotenv
import os
import pandas as pd
import pygsheets
import json

load_dotenv()

True

In [3]:
service_account = pygsheets.authorize(service_account_file='JSONs\\spreadsheet-automator-449222-28b7c907226f.json')

sheet = service_account.open_by_url('https://docs.google.com/spreadsheets/d/1p3ecWHthVRXvAEr900K8YOEt_c2ey98v8ym3I_GTWEA/edit?usp=sharing')


In [4]:
test = sheet.worksheet('title', 'Test')

In [5]:
from dotenv import load_dotenv
load_dotenv()

import os
import requests

API_KEY= os.environ.get('RIOT_API_KEY')

def get_puuid(summonerId=None, gameName=None, tagLine=None, region='americas'):

    if summonerId is not None:
        root_url = f'https://{region}.api.riotgames.com/'
        endpoint = 'lol/summoner/v4/summoners/'
        response = requests.get(root_url+endpoint+summonerId+'?api_key='+API_KEY)
        return response.json()['puuid']
    else:
        root_url = f'https://{region}.api.riotgames.com/'
        endpoint = f'riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}'
        response = requests.get(root_url+endpoint+'?api_key='+API_KEY)
        return response.json()['puuid']

def get_idtag_from_puuid(puuid=None):
    root_url = 'https://americas.api.riotgames.com/'
    endpoint = 'riot/account/v1/accounts/by-puuid/'

    response = requests.get(root_url+endpoint+puuid+'?api_key'+API_KEY)

    id = {
        'gameName' : response.json()['gameName'],
        'tagLine' : response.json()['tagLine']
    }

    return id

In [2]:
def get_ladder(top=None,):
    root = 'https://na1.api.riotgames.com/'
    chall = 'lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    gm = 'lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'
    masters = 'lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'
    chall_response = requests.get(root + chall + '?api_key=' + API_KEY)
    gm_response = requests.get(root + gm + '?api_key=' + API_KEY)
    masters_response = requests.get(root + masters + '?api_key=' + API_KEY)
    chall_df = pd.DataFrame(chall_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    gm_df = pd.DataFrame(gm_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    masters_df = pd.DataFrame(masters_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)

    ladder = pd.concat([chall_df, gm_df, masters_df]).reset_index(drop=True)[0:top]
    ladder = ladder.drop(columns='rank').reset_index(drop=False).rename(columns={'index':'rank'})
    ladder['rank'] = ladder['rank'] + 1
    
    return ladder

def get_match_history(region=None, puuid=None, start=0, count=20):
    root_url = f'https://{region}.api.riotgames.com/'
    endpoint = f'lol/match/v5/matches/by-puuid/{puuid}/ids'
    query_params = f'?queue=450&type=normal&start={start}&count={count}'

    response = requests.get(root_url + endpoint + query_params +'&api_key=' + API_KEY)


    return response.json()

def get_match_data_from_id(region=None, matchId=None):
    root_url = f'https://{region}.api.riotgames.com/'
    endpoint = f'/lol/match/v5/matches/{matchId}'

    response = requests.get(root_url+endpoint+'?api_key='+API_KEY)

    return response.json()

In [7]:
get_ladder(300)

,rank,summonerId,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,1,i3bKspa2lFG7JkPnxGylCdQ94elts1D4emgQqFiJSEtr7sw,1614,116,79,True,False,False,True
1,2,yKKXTvcFPlTL8qIFO8j7h5pIsihllh92VliC1m1IlO7IFcM,1603,124,83,True,False,False,True
2,3,U0NRKUez6uuiYSLieKIlT_LL25pd-vaA0kjdsIMMJLvjsZ8,1569,108,69,True,False,False,False
3,4,xT1TVF9uPs7iHEJ3lbfyCL2EJwaou6Bu57fbwWJttQZE4Y...,1432,91,53,False,False,False,True
4,5,zTDVfcGmuxQnHL5PIc0dfCIe6t_nAVgWMJOR0dCispnn1Zyg,1424,82,48,False,False,False,False
...,...,...,...,...,...,...,...,...,...
295,296,reFDL40FSP8_3f_k8zNVeHl_ZebK8ZJ4eAnwUCqKMMq_d2...,470,63,54,False,False,False,False
296,297,Vu_6ftY4n8get971XobsxmHnD0eKwlED36nxIt1YEPPxv40,462,79,70,False,False,True,False
297,298,TEE0uW__1T7WGtw9fzeSzhRG5-LC7HfbSY0To9MOjhigsis,453,106,86,False,False,True,False
298,299,NQOja9XPSjVFQdh5FRKs1sRqQbDv1Lmm8_OVeOueu1VR5bvE,435,131,112,False,False,True,False


In [7]:
get_puuid(gameName="denial", tagLine = "viego")

'podsNUGPSvp7NFmwK5CN2b7-kuJiDeciSCveMXtuAM8K8qYkGt71g815t0R6x5LHgpTgtpSuFs1cBQ'

In [9]:
get_match_history(region="americas", puuid="5kXhUQSJDvpYKmmsKN5OzSvDhel0y2YxGrgCMsl_CabC_8NXPZTCJTi5KYWSxa62sIrPEVHaQ8FVyw")

['NA1_5221819613',
 'NA1_5216854883',
 'NA1_5216832915',
 'NA1_5215873552',
 'NA1_5215859385',
 'NA1_5215844362',
 'NA1_5215824821',
 'NA1_5215806452',
 'NA1_5215787352',
 'NA1_5214993103',
 'NA1_5214977325',
 'NA1_5214956912',
 'NA1_5214933780',
 'NA1_5212746743',
 'NA1_5212728948',
 'NA1_5211161275',
 'NA1_5211160180',
 'NA1_5209991702',
 'NA1_5209975972',
 'NA1_5207756536']

In [ ]:
get_match_data_from_id(region="americas", matchId="NA1_5216832915")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [8]:
matches = get_match_history(region="americas", puuid='podsNUGPSvp7NFmwK5CN2b7-kuJiDeciSCveMXtuAM8K8qYkGt71g815t0R6x5LHgpTgtpSuFs1cBQ', count=100)
with open('snowballs.json', 'r') as file:
    snowballs = json.load(file)
friendsList = {'denial#viego', '23ejd#NA1', 'NightIsShadow#NA1', 'demon slayer yi#NA1', 'Alias Mudd#NA1', 'Azirs Pantaloons#Azir', 'Kiramdzz#1204', 'OriKitty#meow'}
flag = False
count = 0
for x in matches:
    if flag:
        break
    match_data = get_match_data_from_id(region = 'americas', matchId = x)
    for player in match_data["info"]["participants"]:
        puuid = player['puuid']
        index = -1
        for i, p in enumerate(snowballs):
            if p["puuid"] == puuid:
                index = i
                break
        snowballsHit=0
        name = player['riotIdGameName']
        tagline = player['riotIdTagline']
        profileIconId = 0
        url = f'https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}?api_key={API_KEY}'
        profile = (requests.get(url))
        if profile.status_code == 200:
            profileIconId = (profile.json())['profileIconId']
        else:
            print(profile.status_code)
            flag = True
            break
        url = f'https://na1.api.riotgames.com/lol/challenges/v1/player-data/{puuid}?api_key={API_KEY}'
        challengeData = (requests.get(url))
        if challengeData.status_code == 200:
            challengeData = (challengeData.json())['challenges']
            challenge_id_to_find = 101203
            value = None
            for j, challenge in enumerate(challengeData):
                if challenge["challengeId"] == challenge_id_to_find:
                    value = challenge["value"]
                    break
            if value is not None:
                    snowballsHit = value
            else:
                print(challengeData.status_code)
                flag = True
                break
            data = {
                    "puuid" : puuid,
                    "name" : name,
                    "tagline" : tagline,
                    "snowballsHit" : snowballsHit,
                    "profileIconId" : profileIconId
                    }
            if (index == -1):
                snowballs.append(data)
            else:
                snowballs[index] = data
with open('snowballs.json', 'w') as file:
    json.dump(snowballs, file)
print(snowballs)


429
[{'puuid': 'vcsWJvj3TLLZGzY-O5T-whqWRBxwG9Ycq9y3y2adpMNBVP8Z7K5DXDlaqkD261lF5m8JfH4SYPNtQg', 'name': 'MeowSick', 'tagline': 'NA1', 'snowballsHit': 72118.0, 'profileIconId': 5414}, {'puuid': 'EyI0pXJJ-iX6mMb3WrNqSk1AOI2-deEDn7r7BjRMAsANlsV9RQDDx1qE3lfJLfSDsN4T6jwgPjFD_g', 'name': 'StandardSumo', 'tagline': 'NA1', 'snowballsHit': 24811.0, 'profileIconId': 5621}, {'puuid': 'gpHy6KlEWyQ6SMPJdh5VnkarlI4_mEEZQMbrsQfF5nADGneBTsvcLrlAt7-WLBaI0XGukDeHw1ZWnQ', 'name': 'Pho Lyfe', 'tagline': 'NA1', 'snowballsHit': 18713.0, 'profileIconId': 6706}, {'puuid': 'Cj1uKvR6cctyG7Vg1xjpAlViuEoXp56iHC-wd0NhSweMvkIVFzrOBn9Ck3RjEx-kPIrrQ02DbA2LMA', 'name': 'Soulpie', 'tagline': 'NA1', 'snowballsHit': 16321.0, 'profileIconId': 6503}, {'puuid': 'ua5o6n1o3E84un2x75v-Z5Ytfd_IlJtlmnWcdBZ2mZT6V9eg8GHuk7BVIJdLfP9WEVOBawVACRvZ9Q', 'name': 'Madmax1213', 'tagline': 'NA1', 'snowballsHit': 13510.0, 'profileIconId': 506}, {'puuid': '4LALnnnKaWbpp0AUCCWTpDrqLJxctIhRUoDTHGeN7tIwIpSwVaDkznDOWgn2SPRPnW9y3ubdzWfsvA', 'nam

In [9]:
with open('snowballs.json', 'r') as file:
    snowballs = json.load(file)
snowballs = sorted(snowballs,  key=lambda x: x['snowballsHit'], reverse=True)
with open('snowballs.json', 'w') as file:
    json.dump(snowballs, file)